# Test Existing VAZHI Models

Before creating new training runs, test which existing models still work.

**Models to Test:**
1. `CryptoYogi/vazhi-qwen3-lora-best` - Best from v0.8 cycle training
2. `CryptoYogi/qwen3-0.6b-vazhi` - Latest (v3.3 - likely broken)
3. `CryptoYogi/vazhi-qwen3-lora-cycle-6` - Last cycle checkpoint

In [ ]:
!pip install -q transformers peft accelerate torch

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

def test_model(model_name, is_lora=True):
    print(f"\n{'='*60}")
    print(f"Testing: {model_name}")
    print('='*60)
    
    try:
        if is_lora:
            # Load base model + LoRA adapter
            base_model = "Qwen/Qwen3-0.6B"
            print(f"Loading base: {base_model}")
            model = AutoModelForCausalLM.from_pretrained(
                base_model,
                torch_dtype=torch.float16,
                device_map="auto",
                trust_remote_code=True
            )
            print(f"Loading LoRA: {model_name}")
            model = PeftModel.from_pretrained(model, model_name)
            tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
        else:
            # Load merged model directly
            print(f"Loading merged model: {model_name}")
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=torch.float16,
                device_map="auto",
                trust_remote_code=True
            )
            tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        
        model.config.use_cache = True
        
        # Test prompts
        test_prompts = [
            "வணக்கம்",
            "தமிழ்நாட்டின் தலைநகரம் என்ன?",
            "2+2 என்ன?",
        ]
        
        system_prompt = "நீங்கள் VAZHI (வழி), தமிழ் மக்களுக்கான AI உதவியாளர்."
        
        for prompt in test_prompts:
            full_prompt = f"<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
            
            inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=100,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id,
                )
            
            response = tokenizer.decode(outputs[0], skip_special_tokens=False)
            if "<|im_start|>assistant" in response:
                response = response.split("<|im_start|>assistant")[-1]
                response = response.split("<|im_end|>")[0].strip()
            
            print(f"Q: {prompt}")
            print(f"A: {response[:200]}")
            print("-" * 40)
        
        # Check if output looks valid
        if "system" in response.lower() or len(response) < 5:
            print("\n❌ MODEL APPEARS BROKEN")
        elif any(ord(c) > 0x0B00 and ord(c) < 0x0BFF for c in response):
            print("\n✅ MODEL PRODUCES TAMIL OUTPUT")
        else:
            print("\n⚠️ MODEL OUTPUT MAY BE PROBLEMATIC")
            
    except Exception as e:
        print(f"❌ Error loading model: {e}")
    
    # Clean up
    del model
    torch.cuda.empty_cache()

In [ ]:
# Test vazhi-qwen3-lora-best (v0.8 best checkpoint)
test_model("CryptoYogi/vazhi-qwen3-lora-best", is_lora=True)

In [ ]:
# Test vazhi-qwen3-lora-cycle-6 (last cycle)
test_model("CryptoYogi/vazhi-qwen3-lora-cycle-6", is_lora=True)

In [ ]:
# Test qwen3-0.6b-vazhi (latest merged - likely broken)
test_model("CryptoYogi/qwen3-0.6b-vazhi", is_lora=True)

## Results

Based on the tests above:

| Model | Status | Notes |
|-------|--------|-------|
| vazhi-qwen3-lora-best | ? | Best from v0.8 |
| vazhi-qwen3-lora-cycle-6 | ? | Last cycle |
| qwen3-0.6b-vazhi | ? | Latest (v3.3) |

**Next Steps:**
- If lora-best works: Continue training from there
- If all broken: Use v3.4 notebook with Qwen3-0.6B-Base